In [1]:


import astropy
print('astropy',astropy.__version__ )
from spectral_cube import SpectralCube      # This is a handy package for working with 3D data cubes
from astropy.coordinates import SkyCoord
from reproject import reproject_interp      
from reproject.mosaicking import find_optimal_celestial_wcs 
import regions
import reproject
print('reproject',reproject.__version__)
import spectral_cube
print('spectral_cube',spectral_cube.__version__)
import numpy as np                          
import pylab                                
import matplotlib 
import matplotlib.gridspec as gridspec                                                                                             
import scipy
import astropy.io.fits as fits                                                          
from astropy.wcs import WCS                 
from astropy import units as u              
import pyspeckit as psk   
import astrodendro
import matplotlib.colors as colors
from matplotlib import pyplot as plt
# Suppress warnings we don't care about:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
%matplotlib inline

#%matplotlib widget

astropy 5.1.dev153+gb740594dc
reproject 0.8
spectral_cube 0.6.1.dev22+g003ef16


In [2]:
files = pylab.array(['NGC253.HCO+_4-3.pbcor.fits','NGC253.HCN_4-3.pbcor.fits','NGC253.H13CN_4-3.pbcor.fits','HCN_J1-0.cube.fits','HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits'])
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[int(len(sc)*.2):int(len(sc)*.8),raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab.to(u.K)
    
    
    
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
        NGC_HCO_Plus.write('NGC_HCO_Plus.cube.K-kms.pbcor.fits',overwrite=True)        
    if i==1:
        NGC_HCN = sc_K_kms.unmasked_copy()
        NGC_HCN.write('NGC253_HCN.cube.reproject.K-kms.pbcor.fits',overwrite=True)
        del NGC_HCN
    if i==2:
        
        NGC_H13CN = sc_K_kms.unmasked_copy()#This one needs reprojection

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_H13CN.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
del NGC_HCO_Plus #rem from mem
NGC_H13CN_reproject = NGC_H13CN.reproject(HCOPH,use_memmap=True)  
del NGC_H13CN #rem from mem
del HCOPH
NGC_HC13N_reproject_interp = NGC_H13CN_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del NGC_H13CN_reproject
NGC_HC13N_reproject_interp.write('NGC_H13CN.cube.reproject.K-kms.pbcor.fits',overwrite=True)      
del NGC_HC13N_reproject_interp         
print("done")        

ValueError: An output array of a different type than the input array was specified, which will create an undesired duplicate copy of the input array in memory.

In [ ]:
#The first three cubes haves been reporjected

In [3]:
files = pylab.array(['NGC253.HCO+_4-3.pbcor.fits','NGC253.HCN_4-3.pbcor.fits','NGC253.H13CN_4-3.pbcor.fits','HCN_J1-0.cube.fits','HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits'])
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[int(len(sc)*.28):int(len(sc)*.73),raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab.to(u.K)
    
    
    
    if i==3:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Need new vel axis because vrad!=2vrad
        NGC_HCN_J1 = sc_K_kms.unmasked_copy()
    if i==4:
        NGC_HCOP_J1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_HCOP_J1.allow_huge_operations=True
HCNH = NGC_HCN_J1.header
del NGC_HCN_J1 #rem from mem
NGC_HCOP_J1_reproject = NGC_HCOP_J1.reproject(HCNH,use_memmap=True)  
del NGC_HCOP_J1 #rem from mem
del HCNH
NGC_HCOP_J1_reproject_interp = NGC_HCOP_J1_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del NGC_HCOP_J1_reproject
NGC_HCOP_J1_reproject_interp.write('NGC_HCOP_J1_0.cube.reproject.K-kms.pbcor.fits',overwrite=True)      
del NGC_HCOP_J1_reproject_interp        
    


print("done")        

done


In [2]:
files = pylab.array(['NGC253.HCO+_4-3.pbcor.fits','NGC253.HCN_4-3.pbcor.fits','NGC253.H13CN_4-3.pbcor.fits','HCN_J1-0.cube.fits','HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits'])
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[int(len(sc)*.28):int(len(sc)*.73),raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab.to(u.K)
    
    
    
    if i==3:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Need new vel axis because vrad!=2vrad
        NGC_HCN_J1 = sc_K_kms.unmasked_copy()
    if i==5:
        NGC_H13CN_J1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_H13CN_J1.allow_huge_operations=True
HCNH = NGC_HCN_J1.header
del NGC_HCN_J1 #rem from mem
NGC_H13CN_J1_reproject = NGC_H13CN_J1.reproject(HCNH,use_memmap=True)  
del NGC_H13CN_J1 #rem from mem
del HCNH
NGC_H13CN_J1_reproject_interp = NGC_H13CN_J1_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del NGC_H13CN_J1_reproject
NGC_H13CN_J1_reproject_interp.write('NGC_H13CN_J1_0.cube.reproject.K-kms.pbcor.fits',overwrite=True)  
del NGC_H13CN_J1_reproject_interp        
    

print("done")        

done


In [5]:
files = pylab.array(['NGC253.HCO+_4-3.pbcor.fits','NGC253.HCN_4-3.pbcor.fits','NGC253.H13CN_4-3.pbcor.fits','HCN_J1-0.cube.fits','HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits'])
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[int(len(sc)*.28):int(len(sc)*.73),raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab.to(u.K)
    
    
    
    if i==3:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Need new vel axis because vrad!=2vrad
        NGC_HCN_J1 = sc_K_kms.unmasked_copy()
    if i==6:
        NGC_CO_J3_2 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_CO_J3_2.allow_huge_operations=True
HCNH = NGC_HCN_J1.header
del NGC_HCN_J1 #rem from mem
NGC_CO_J3_2_reproject = NGC_CO_J3_2.reproject(HCNH,use_memmap=True)  
del NGC_CO_J3_2 #rem from mem
del HCNH
NGC_CO_J3_2_reproject_interp = NGC_CO_J3_2_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del NGC_CO_J3_2_reproject
NGC_CO_J3_2_reproject_interp.write('NGC_CO_J3_2.cube.reproject.K-kms.pbcor.fits',overwrite=True)      
del NGC_CO_J3_2_reproject_interp        
    
    
print("done")     

done


In [6]:
files = pylab.array(['NGC253.HCO+_4-3.pbcor.fits','NGC253.HCN_4-3.pbcor.fits','NGC253.H13CN_4-3.pbcor.fits','HCN_J1-0.cube.fits','HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits'])
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[int(len(sc)*.28):int(len(sc)*.73),raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab.to(u.K)
    
    
    
    if i==3:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Need new vel axis because vrad!=2vrad
        NGC_HCN_J1 = sc_K_kms.unmasked_copy()
    if i==7:
        CCH_N1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
CCH_N1.allow_huge_operations=True
HCNH = NGC_HCN_J1.header
del NGC_HCN_J1 #rem from mem
CCH_N1_reproject = CCH_N1.reproject(HCNH,use_memmap=True)  
del CCH_N1 #rem from mem
del HCNH
CCH_N1_reproject_interp = CCH_N1_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del CCH_N1_reproject
CCH_N1_reproject_interp.write('CCH_N1.cube.reproject.K-kms.pbcor.fits',overwrite=True)   
del CCH_N1_reproject_interp        
    
    

    
print("done")     

done


In [2]:
files = pylab.array(['NGC253.HCO+_4-3.pbcor.fits','NGC253.HCN_4-3.pbcor.fits','NGC253.H13CN_4-3.pbcor.fits','HCN_J1-0.cube.fits','HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits',"HC3N_11_10.fits"])
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[int(len(sc)*.28):int(len(sc)*.73),raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab.to(u.K)
    
    
    
    if i==3:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Need new vel axis because vrad!=2vrad
        NGC_HCN_J1 = sc_K_kms.unmasked_copy()
    if i==8:
        HC3N_J11 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
HC3N_J11.allow_huge_operations=True
HCNH = NGC_HCN_J1.header
del NGC_HCN_J1 #rem from mem
HC3N_J11_reproject = HC3N_J11.reproject(HCNH,use_memmap=True)  
del HC3N_J11 #rem from mem
del HCNH
HC3N_J11_reproject_interp = HC3N_J11_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del HC3N_J11_reproject
HC3N_J11_reproject_interp.write('HC3N_J11.cube.reproject.K-kms.pbcor.fits',overwrite=True) 
del HC3N_J11_reproject_interp        
    
print("done")

done


In [12]:

      
files = pylab.array(['HCN_J1-0.cube.fits', "H13CO_Plus_1_0.fits"])


for i,file in enumerate(files):
                    
    sc = SpectralCube.read(file)  
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[int(len(sc)*.28):int(len(sc)*.73),raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab.to(u.K)
    
    
    
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Need new vel axis because vrad!=2vrad
        NGC_HCN_J1 = sc_K_kms.unmasked_copy()
    if i==1:
        H13CoPJ1_0 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
H13CoPJ1_0.allow_huge_operations=True
HCNH = NGC_HCN_J1.header
del NGC_HCN_J1 #rem from mem
H13CoPJ1_0_reproject = H13CoPJ1_0.reproject(HCNH,use_memmap=True)  
del H13CoPJ1_0 #rem from mem
del HCNH
H13CoPJ1_0_reproject_interp = H13CoPJ1_0_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del H13CoPJ1_0_reproject
H13CoPJ1_0_reproject_interp.write('H13CoPJ1_0.cube.reproject.K-kms.pbcor.fits',overwrite=True)      
del H13CoPJ1_0_reproject_interp        
    
                    
         
print("done")

done


In [ ]:
### files = pylab.array(['HCN_J1-0.cube.fits', "SiO_2_1.fits"])


for i,file in enumerate(files):
                    
    sc = SpectralCube.read(file)  
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[int(len(sc)*.28):int(len(sc)*.73),raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab.to(u.K)
    
    
    
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Need new vel axis because vrad!=2vrad
        NGC_HCN_J1 = sc_K_kms.unmasked_copy()
    if i==1:
        SiJ2_1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
SiJ2_1.allow_huge_operations=True
HCNH = NGC_HCN_J1.header
del NGC_HCN_J1 #rem from mem
SiJ2_1_reproject = SiJ2_1.reproject(HCNH,use_memmap=True)  
del SiJ2_1 #rem from mem
del HCNH
SiJ2_1_reproject_interp = SiJ2_1_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del SiJ2_1_reproject
SiJ2_1_reproject_interp.write('SiJ2_1.cube.reproject.K-kms.pbcor.fits',overwrite=True)       

del SiJ2_1_reproject_interp        
    

    
    
    
print("done")